In [1]:
from helper_optflow2 import *
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import random
import pdb
import numpy as np
import math
import re
import os

In [2]:
class VidNet(nn.Module):
    def __init__(self,method,num_classes,input_features,classifier_size,timesteps_vid,optical_flow):
        super(VidNet, self).__init__()
#         self.num_timesteps = num_timesteps
        self.optical_flow = optical_flow
        self.timesteps_vid = timesteps_vid
        self.hidden_dim = 100
        self.method = method
        self.num_classes = num_classes
        self.input_features = input_features
        self.classifier_size = classifier_size
        if self.optical_flow:
            self.hidden = self.init_hidden()
        if self.method=='slow':
            self.mult1 = (self.timesteps_vid//4)//2
            self.mult2 = 2
        else:
            self.mult1 = 1
            self.mult2 = 1
        self.rnn = nn.RNN(12,self.hidden_dim,2,batch_first=True)
        self.layers = nn.Sequential(
            # in, out, stride, padding
            # in, out, stride, padding
            nn.Conv2d(self.input_features,96,11,stride = 3,padding = 2),
            nn.BatchNorm2d(96),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2,2),
            nn.Dropout(.25),
            nn.Conv2d(96*self.mult1,256,5,stride = 1,padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2,2),
            nn.Dropout(.25),
            nn.Conv2d(256*self.mult2,384*self.mult2,3,stride = 1,padding=2),
            nn.Conv2d(384*self.mult2,384,3,stride = 1,padding=2),
            nn.Conv2d(384,256,3,stride = 1,padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3,2))
        self.classifiers =nn.Sequential(
            nn.Linear(9216*self.classifier_size + self.timesteps_vid*self.hidden_dim,4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096,4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096,self.num_classes)
#             nn.Softmax() # maybe make delete this and try F.log_softmax() in forward
            )

    def init_hidden(self):
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        # self.hidden = Variable(torch.zeros(1, self.batch_size, self.hidden_dim).cuda())
        self.hidden =  torch.zeros(2, 10, self.hidden_dim)

    def forward(self,inputs,optflow_in):
        if self.method == 'single' or self.method =='early' or self.method == 'conv_pool':
            x = inputs
            for layer in self.layers:
                x = layer(x)

        elif self.method == 'late':
            x = torch.Tensor(0,0)
            for xind in range(inputs.size(1)):

                xin = inputs[:,xind,:,:].unsqueeze(1)

            # x1,x2 = input
                for layer in self.layers:
                    xin = layer(xin)
                x = torch.cat((x,xin),1).cuda()

        elif self.method == 'slow':
            x = torch.Tensor(0,0).cuda()
            xout = torch.Tensor(0,0).cuda()
            for xind in np.arange(0,inputs.size(1),4):
                xin = inputs[:,xind:xind+4,:,:]
            # x1,x2,x3,x4 = input
                for layer in self.layers[0:5]:
                    xin = layer(xin)
                xout = torch.cat((xout,xin),1)
            xa = xout[:,:int(xout.shape[1]/2),:,:]
            xb = xout[:,int(xout.shape[1]/2):,:,:]
            for layer in self.layers[5:10]:
                xa = layer(xa)
                xb = layer(xb)
            x = torch.cat((xa,xb),1)
            for layer in self.layers[10:]:
                x = layer(x)

        else:
            print('Define Method')
            x = inputs
#             break

        x = x.view(x.size(0),-1)
        if self.optical_flow:
            output, self.hidden = self.rnn(optflow_in,self.hidden)
            rnn_out = output.contiguous().view(output.size(0),-1)
            x = torch.cat((x,rnn_out),1)
        for classifier in self.classifiers:
            x = classifier(x)
        x = F.log_softmax(x)
        return x


In [17]:
class Trainer():
    def __init__(self,net,dic,datapath,timesteps,epochs,
                 optimizer,criterion,batch_size,color,random_sample, skip,
                 optical_flow):
        self.dic = dic
        self.datapath = datapath
        self.optical_flow = optical_flow
        self.method = method
        self.epochs = epochs
        self.optimizer = optimizer
        self.criterion = criterion
        self.timesteps = timesteps
        self.net = net
        self.batch_size = batch_size
        self.color = color
        self.random_sample = random_sample
        self.skip = skip
        self.rng = np.random.RandomState(123456)
        self.proj = self.rng.randn(12,2*110*110).astype(np.float32)

    def get_accuracy(self,outputs,labels):
        _, predicted = torch.max(outputs.data, 1)
        correct = float(((predicted == labels.detach()).sum()).item())
        total = labels.size(0)
        return correct, total

    def train_test(self,path,train):
        loss_vec = []
        acc_vec = []
        # net = VidNet(self.timesteps,self.method)
#         content_t = []
        paths = os.listdir(path)
        paths = [path[2:] for path in paths if path[0]=='.']
        pt_test = paths[::4]
        pt_train = list(set(paths) - set(pt_test))
        if train:
            paths = pt_train
        else:
            paths = pt_test
#         random.shuffle(paths)
        labels = [re.findall('_([^_]*)_', datastr)[0] for datastr in paths]
#         x = list(set(labels))
        self.dic = dict(zip(x, list(range(0,len(x)))))
        num_labels = [dic[v] for v in labels]

#         paths = [re.findall('"([^"]*)"', c)[0] for c in content_t]
        running_loss = 0
        correct = 0
        total = 0
        for bs in range(self.batch_size):
            b_targets = num_labels[bs*self.batch_size:(bs*self.batch_size)+self.batch_size]
            b_path = paths[bs*self.batch_size:(bs*self.batch_size)+self.batch_size]
            frame_array=[]
            # import pdb; pdb.set_trace()
            flpts_array = []
            for vid_name in b_path:
#                 if vid_name[0]=='.':
#                     vid_name = vid_name[2:]
                if self.optical_flow:
                    try:
                        frame, flpts = video3d(self.datapath + vid_name, (110,110,self.timesteps),
                                            color=self.color,skip=self.skip,rand=self.random_sample,
                                            optical_flow=self.optical_flow,proj = self.proj)
                    except:
                        import pdb; pdb.set_trace()
                    frame = torch.Tensor(np.array(frame))
                    # (x_new,y_new),(x_old,y_old)  N x 2, N x 2
                    # new = flpts[0][0]

                    flpts = np.array(flpts)
                else:
                    frame = torch.Tensor(np.array(video3d(self.datapath + vid_name,
                                                                     (110,110,self.timesteps),color=self.color,
                                                                     skip=self.skip,rand=self.random_sample,
                                                                     optical_flow=self.optical_flow)))
                frame_array.append(frame)
                flpts_array.append(flpts)
            # import pdb; pdb.set_trace()
            flpts_ar = torch.Tensor(np.array(flpts_array))
            torch_frames = torch.cat(frame_array,0)
            self.optimizer.zero_grad()
            net.init_hidden()
            if self.optical_flow:
                outputs = self.net(torch_frames,flpts_ar)
            else:
                outputs = self.net(torch_frames)
            targets = torch.LongTensor(b_targets)
            loss = self.criterion(outputs, targets)
#             import pdb; pdb.set_trace()
            if train:
                loss.backward()
                self.optimizer.step()
            running_loss += loss.detach().numpy()
#             import pdb; pdb.set_trace()
            correct_i, total_i = self.get_accuracy(outputs, targets)
            correct += correct_i
            total += total_i
#         import pdb; pdb.set_trace()
        loss_vec.append(running_loss/(len(labels))/self.batch_size)
        acc_vec.append(correct/total)
        return loss_vec, acc_vec

    def run_epochs(self,save_name):
        e_train_loss = []
        e_train_acc = []
        e_test_loss = []
        e_test_acc = []
        for i in range(self.epochs):
            train_loss, train_acc = self.train_test(self.datapath, train=True)
            test_loss, test_acc = self.train_test(self.datapath, train=False)
            e_train_loss.append(train_loss)
            e_test_loss.append(test_loss)
            e_train_acc.append(train_acc)
            e_test_acc.append(test_acc)
            torch.save(self.net.state_dict(),save_name + '.pt')
            torch.save(self.optimizer.state_dict(),save_name + '_opt.pt')
            with open(save_name + '_trainloss.csv','a') as f:
                f.write(str(train_loss))
            with open(save_name + '_trainacc.csv','a') as f:
                f.write(str(train_acc))
            with open(save_name + '_testloss.csv','a') as f:
                f.write(str(test_loss))
            with open(save_name + '_testacc.csv','a') as f:
                f.write(str(test_acc))
            print('Epoch ' + str(i) + ':  Train Loss: ' + str(train_loss) + '; Test Loss: ' + str (test_loss))
            print('Epoch ' + str(i) + ':  Train Accuracy: ' + str(train_acc) + '; Test Accuracy: ' + str (test_acc))
        return e_train_loss, e_test_loss, e_train_acc, e_test_acc


In [ ]:
base_path = 'VideoData/'
paths = os.listdir(base_path)
paths = [path[2:] for path in paths if path[0]=='.']
labels = [re.findall('_([^_]*)_', datastr)[0] for datastr in paths]
x = list(set(labels))
dic = dict(zip(x, list(range(0,len(x)))))
method = 'conv_pool'
num_epochs = 50
torch.set_default_tensor_type('torch.FloatTensor')
color = False
random_sample = False
skip = True
load = False
save_name = 'NEWDATAoptflow_conv_rand_noskip'

if color:
    input_features = 3
else:
    input_features = 1

if method == 'single':
    timesteps_vid = 1
    timesteps_net = 1
    net = VidNet(method,6,input_features*timesteps_net,timesteps_vid,timesteps_vid,optical_flow=True)
elif method=='conv_pool':
    timesteps_vid = 16
    timesteps_net = 16
    net = VidNet(method,6,input_features*timesteps_net,1,timesteps_vid,optical_flow=True)
elif method == 'late':
    timesteps_vid = 8
    timesteps_net = 1
    net = VidNet(method,6,input_features*timesteps_net,timesteps_vid,timesteps_vid,optical_flow=True)
elif method == 'slow':
    timesteps_vid = 16
    timesteps_net = 4
    net = VidNet(method,6,input_features*timesteps_net,1,timesteps_vid,optical_flow=True)
else:
    print('define valid method')
if load:
    net.load_state_dict(torch.load(save_name + '.pt'))
    net.cuda()
    print(next(net.parameters()).is_cuda)
#     for statet in net.state.values():
#         for h,g in statet.items():
#             if torch.is_tensor(g):
#                 statet[h] = g.cuda()
    optimize = optim.Adam(net.parameters(), lr = 0.0001, weight_decay = .005)
    optimize.load_state_dict(torch.load(save_name + '_opt.pt'))
    for state in optimize.state.values():
        for k, v in state.items():
            if torch.is_tensor(v):
                state[k] = v.cuda()
else:
    optimize = optim.Adam(net.parameters(), lr = 0.0001, weight_decay = .005)
#     net.cuda()
crit = nn.CrossEntropyLoss()
#(self,net,train_path,test_path,vid_path,epochs,optimizer,criterion)
tr = Trainer(net,dic,base_path,timesteps_vid,num_epochs,optimize,crit,10,color,
                random_sample,skip,optical_flow = True)
trl,tstl,tra,tsta = tr.run_epochs(save_name)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:102: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


> <ipython-input-17-1cfb68c10ede>(66)train_test()
-> frame = torch.Tensor(np.array(frame))
(Pdb) c

Program interrupted. (Use 'cont' to resume).
> /datasets/home/69/969/jdawkins/VideoNet/helper_optflow2.py(51)video3d()
-> fin_dummy = np.dot(proj, flow.flatten())


In [25]:
base_path = 'hollywood/annotations/'
vid_path = 'videoclips'
method = 'slow'
num_epochs = 50
torch.set_default_tensor_type('torch.FloatTensor')
color = False
random_sample = True
skip = True
load = True
save_name = '_FULLoptflow_slow_norand_skip'

if color:
    input_features = 3
else:
    input_features = 1

if method == 'single':
    timesteps_vid = 1
    timesteps_net = 1
    net = VidNet(method,6,input_features*timesteps_net,timesteps_vid,timesteps_vid,optical_flow=True)
elif method=='conv_pool':
    timesteps_vid = 16
    timesteps_net = 16
    net = VidNet(method,6,input_features*timesteps_net,1,timesteps_vid,optical_flow=True)
elif method == 'late':
    timesteps_vid = 8
    timesteps_net = 1
    net = VidNet(method,6,input_features*timesteps_net,timesteps_vid,timesteps_vid,optical_flow=True)
elif method == 'slow':
    timesteps_vid = 16
    timesteps_net = 4
    net = VidNet(method,6,input_features*timesteps_net,1,timesteps_vid,optical_flow=True)
else:
    print('define valid method')
if load:
    net.load_state_dict(torch.load(save_name + '.pt'))
    net.cuda()
    print(next(net.parameters()).is_cuda)
#     for statet in net.state.values():
#         for h,g in statet.items():
#             if torch.is_tensor(g):
#                 statet[h] = g.cuda()
    optimize = optim.Adam(net.parameters(), lr = 0.0001, weight_decay = .005)
    optimize.load_state_dict(torch.load(save_name + '_opt.pt'))
    for state in optimize.state.values():
        for k, v in state.items():
            if torch.is_tensor(v):
                state[k] = v.cuda()
else:
    optimize = optim.Adam(net.parameters(), lr = 0.0001, weight_decay = .005)
    net.cuda()
crit = nn.CrossEntropyLoss()
#(self,net,train_path,test_path,vid_path,epochs,optimizer,criterion)
tr = Trainer(net,base_path+'train_clean.txt',base_path+'test_clean.txt',
                vid_path,timesteps_vid,num_epochs,optimize,crit,10,color,
                random_sample,skip,optical_flow = True)
trl,tstl,tra,tsta = tr.run_epochs(save_name)


True


TypeError: __init__() got multiple values for argument 'optical_flow'